<a href="https://colab.research.google.com/github/Animesh-Guchhait/NLP/blob/master/Assignment_03_AnimeshGuchhait_MDS202004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Animesh Guchhait

MDS202004

In [ ]:
# Important libraries
import math
import numpy as np
import pandas as pd
import random

import json
import nltk

import numpy as np
from copy import deepcopy
import re
import sklearn

import os
from operator import itemgetter
from random import shuffle
from tqdm import tqdm
from collections import defaultdict
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec
import logging
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def extract_text(filename):
    '''
    Extract the text from the file name (json file) and
    index the content from paper_id, title, abstract and body_text fields
    Retuns - text of title, abstract and bodt_text
    '''

    file = open(filename)
    body_text = ""
    abstract = ""
    title = ""
    paper_id = ""

    paper_content = json.load(file)

    #get the paper_id
    if 'paper_id' in paper_content:
        paper_id = paper_content['paper_id']
        
    #get the title, if available
    if 'title' in paper_content['metadata']:
        title = paper_content['metadata']['title']
    #get abstract.text, if available
    if 'abstract' in paper_content:
        for abs in paper_content['abstract']:
            abstract = abstract + abs['text']
    if 'body_text' in paper_content:
        for bt in paper_content['body_text']:
            body_text = body_text + bt['text']


   
    return (title + ' ' + abstract + ' ' + body_text + ' ').lower()

In [ ]:
''' Here using this file_formating function , by giving a folder location and json filenames present in 
that folder, we can get extracted files(using extract_text function in this function) as elements of a list
i.e. the first element of the list will be text file which contain the text of the first json file name (like that)'''


def file_formatting(filenames,path):
    fnames =deepcopy(filenames)
    files = []
    c=0
    for filename in fnames:
        #if c>= 3000:
         # break
        try:
            file=extract_text(path+'/'+filename)
            files.append(file)
           # c=c+1
        except:
            ind = filenames.index(filename)
            #print('hoche na')
            filenames.pop(ind)
    return files

In [ ]:
!cp /content/drive/MyDrive/NLP/pdf_json.zip pdf_json.zip  # Unzip the provided zip data in the colab disk 
!unzip -oq pdf_json.zip
print('Unzip Completed')

Unzip Completed


In [ ]:
# getting path and file name of our pdf_json folder where we unzip our given data
path='/content/pdf_json'
filenames = os.listdir(path)

In [ ]:
files=file_formatting(filenames,path)  # Called the file-formating function to get extracted text documents as elements of a list

So,now I have all the text corpus in the 'files'. I am going to build word2vec model using gensim on these file. Here to be noticed that I have not preprocessed that much (just have used lower case) in this assignment compare to others. As we need sentence structures for sentence vectors.

For word2vec model:

 1.  I will add all the text files in a single one.

 2. Then going to use gensim word2vec model to build and save the word2vec model.

In [ ]:
''' I am adding up all the text file in a single text file and 
saving it with the name covid19_corpus.txt to use it in gensim word2vec model '''


def text(filenames,path):
  with open('covid19_corpus.txt','a') as f:
    for x in files:
      f.write(x)
  print('ready')
  


In [ ]:
text(filenames,path)

ready


In [ ]:
''' Gensim word2vec model with vector size 128, minimum count= 20
   window = 4
   The trained model will be saved in drive in the end'''
def compute_word_vectors(): 
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    text = word2vec.Text8Corpus('covid19_corpus.txt')
    model = word2vec.Word2Vec(text, size=128, window=4, min_count=20, workers=4, iter=10)
    model.save('/content/drive/MyDrive/NLP/word2vec_ass3.model.bin')

In [ ]:
compute_word_vectors() # training word2vec model

2022-05-19 04:47:17,289 : INFO : collecting all words and their counts
2022-05-19 04:47:17,294 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-19 04:47:55,512 : INFO : PROGRESS: at sentence #10000, processed 100000000 words, keeping 2519987 word types
2022-05-19 04:48:34,472 : INFO : PROGRESS: at sentence #20000, processed 200000000 words, keeping 4185241 word types
2022-05-19 04:48:45,648 : INFO : collected 4604749 word types from a corpus of 228487036 raw words and 22849 sentences
2022-05-19 04:48:45,650 : INFO : Loading a fresh vocabulary
2022-05-19 04:48:48,402 : INFO : effective_min_count=20 retains 216466 unique words (4% of original 4604749, drops 4388283)
2022-05-19 04:48:48,404 : INFO : effective_min_count=20 leaves 219270587 word corpus (95% of original 228487036, drops 9216449)
2022-05-19 04:48:49,099 : INFO : deleting the raw counts dictionary of 4604749 items
2022-05-19 04:48:49,302 : INFO : sample=0.001 downsamples 29 most-common words
2

Now word2vec model has been built and have saved in the following link:

https://drive.google.com/file/d/1kY0Sa2GIe9hjlFTsuR-UWpS6qP5H39bq/view?usp=sharing

Now to build the sentence collection I am going to use 200 files from the all of my files. I will choose these files randomly.



In [ ]:
random.seed(42)
file_20=random.sample(files,200)  # randomly selecting 200 files 

In [ ]:
'''Here I will tokenize the sentence and will make it for RNN model usable  '''

l=[]
for i in file_20:
  t=[word_tokenize(t) for t in sent_tokenize(i)]
  for i in range(len(t)):
    m=t[i]
    m.pop()       # to delete '.','?' and '!' at the end of each sentence
    t[i]=m
  l.append(t)
  

In [ ]:
del file_20 # to recover space
del files

In [ ]:
''' Now I will keep only those sentence that has length in between 7 to 15 '''
l1=[]
for i in l:
  for j in i:
    s=len(j)
    if s>6:
      if s<=15:
        l1.append(j)
del l

In [ ]:
len(l1) # The number of sentence I have got

3735

In [ ]:
# Saving these sentence for future use 
from numpy import save
save('/content/drive/MyDrive/NLP/sentence.npy', np.array(l1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


Collection of sentence 

https://drive.google.com/file/d/1FxzvGSxZjAO30Qx6z0HNf7x11pFbeLYU/view?usp=sharing

Now I have tokenize sentences and word2vec model is ready. So, now we can go for RNN for sentence vectors. 

In [ ]:
model = Word2Vec.load('/content/drive/MyDrive/NLP/word2vec_ass3.model.bin')  # Load the word2vec model

In [ ]:
# Loading l1 that I have saved earlier. In case you have disconnected, only for then you have to ran this one
l1=np.load('/content/drive/MyDrive/NLP/sentence.npy',allow_pickle=True)
l1=list(l1)

RNN model for sentence vector

In [ ]:
''' it will generate random initialization of required parametes for RNN model '''

def rnn_parameters(U_dim=(192,192),V_dim=(192,128),h_dim=192):
  U = np.random.normal(0,0.01,U_dim)
  V = np.random.normal(0,0.01,V_dim)
  s0 = np.random.normal(0,0.01,h_dim)
  return(U,V,s0)

In [ ]:
# To get tanh activation function
def tanh_function(vec):
  return np.tanh(vec)

In [ ]:
''' if you provide a collection of words of a sentence to it, (obviously with word2vec model) 
   it will generate sentence vector for that sentecne 
   
   here I am taking sentence vector dimension =192
                    word vector dimension =128
                    
'''
def RNN_sentence_2Vec(model,sentence,h_dim=192):
  np.random.seed(42)
  U,V,s0 = rnn_parameters()  # Random initialization of parameters

  for word in sentence:
    try:
      s0=tanh_function(np.matmul(V,np.array([model.wv[word]])[0])+np.matmul(U,s0)) # RNN loop structure
    except:
      s0=tanh_function(np.zeros(h_dim)+np.matmul(U,s0)) # if a word is not present in word2vec model (because of lower frequancy of word)
                                                        # make that word vector as zero vector
  return(s0)


In [ ]:
''' This function will call RNN_sentence_2Vec to get sentence vector and will store those
in a list and will return that sentence vector matrix as return '''

def sentence_vec_matrix(l1,model):
  sentence_vec_matrix=[]
  for i in tqdm(l1):
    sentence_vec_matrix.append(list(RNN_sentence_2Vec(model,i)))
  return sentence_vec_matrix

In [ ]:
sentence_vec=sentence_vec_matrix(l1,model)

100%|██████████| 3735/3735 [00:17<00:00, 212.02it/s]


In [ ]:
index=[i for i in range(3735)]  # Generate the index of each sentence 

In [ ]:
''' Similarity matrix (Using cosine similarity) will give 5 most related sentence to a given sentence 
    Here I have provided some index of sentences and it will give related sentences to those sentences  '''


def similarity_matrix(my_sentence_index,index,sentence_vec,l1):
  c=0
  for i in  my_sentence_index:
    l2=[[-99,None],[-99,None],[-99,None],[-99,None],[-99,None],[-99,None],[-99,None],[-99,None],[-99,None],[-99,None]]
    for j in index:
      if i == j:
        continue
      similarity=cosine_similarity(np.array([sentence_vec[i]]),np.array([sentence_vec[j]])) # cosine similarity
      if similarity > l2[0][0]:
        l2[0]=[similarity,' '.join(l1[j])]
        l2.sort()
    l2.sort()
    c=c+1
    print(c)
    print('Related sentence to -- ',' '.join(l1[i]))
    print()
    l2.reverse()
   
    for i in l2:
      print(i)
    print()


In [ ]:
# I have gone through some of sentence in my sentence collection,and have chosen arbitrarily some index of those sentence related to
# covid , diseases and hospital.
my_sentence_new_index=[10,55,57,68,100,3678,3666,3664,3659,3637]

In [ ]:
similarity_matrix(my_sentence_new_index,index,sentence_vec,l1)

1
Related sentence to --  all f-statistics are significant at 1 %

[array([[1.]]), '2. all f-statistics are significant at 1 %']
[array([[0.99483268]]), 'adenovirus disease is associated with a fatality rate of 30-50 %']
[array([[0.99478238]]), 'this prospective trial included 1421 patients , and mortality at 3 months was 6.3 %']
[array([[0.99471529]]), 'the point estimate of the runny nose was found as 0.0 %']
[array([[0.9946105]]), 'the overall tumour-delivery efficiency was 15.8 %']
[array([[0.99451433]]), 'we found that both chlorpromazine and bch inhibited laam tc-cqd internalization by around 80 %']
[array([[0.99443134]]), 'the authors reported an accuracy of 95.7 % and a sensitivity of 90 %']
[array([[0.99431999]]), '11 and reported prevalence of chemosensory dysfunction from 5,1 % to 19.4 %']
[array([[0.994254]]), 'under this program , vaccination levels reached 80 %']
[array([[0.99418223]]), 'in this study , the authors reported an accuracy of 92.6 %']

2
Related sentence to -

In [ ]:
# it will normalize the given vector. 
# it is used to normalize sentence vector which I am going to use to find l2 distance.
def normalizing(sentence_vec):
  l=np.array(sentence_vec)
  norm=np.linalg.norm(l, axis=1)
  for i in range(len(l)):
    l[i]=l[i]/norm[i]
  return(l)


In [ ]:
normalized_sentence_vec=normalizing(sentence_vec) # normalizing sentence vectors.

In [ ]:
''' Using l2_distance I am going to find closer sentence to a given sentence. 
   This function will calculate distance of a sentence to other sentences and will
   show top 5 closer sentences as related sentence to the given sentence '''

def l2_distance(my_sentence_index,index,sentence_vec,l1):
    c=0
    for i in  my_sentence_index:
      l2=[[9999,None],[9999,None],[9999,None],[9999,None],[9999,None],[9999,None],[9999,None],[9999,None],[9999,None],[9999,None]]
      for j in index:
        if i == j:
          continue
        distance=np.linalg.norm(sentence_vec[i]-sentence_vec[j]) # l2 distance
        #if distance<5:
        #print(distance)
        
        if distance < l2[-1][0]:
          l2[-1]=[distance,' '.join(l1[j])]
          l2.sort()
      l2.sort()
      c=c+1
      print(c)
      print('Related sentence to -- ',' '.join(l1[i]))
      print()
      
      for i in l2:
        print(i)
      print()

In [ ]:
l2_distance(my_sentence_new_index,index,normalized_sentence_vec,l1)

1
Related sentence to --  all f-statistics are significant at 1 %

[4.161885521513231e-07, '2. all f-statistics are significant at 1 %']
[0.10165946936406484, 'adenovirus disease is associated with a fatality rate of 30-50 %']
[0.10215299475797703, 'this prospective trial included 1421 patients , and mortality at 3 months was 6.3 %']
[0.10280767063075219, 'the point estimate of the runny nose was found as 0.0 %']
[0.10382194909018934, 'the overall tumour-delivery efficiency was 15.8 %']
[0.10474418396744783, 'we found that both chlorpromazine and bch inhibited laam tc-cqd internalization by around 80 %']
[0.10553354772305933, 'the authors reported an accuracy of 95.7 % and a sensitivity of 90 %']
[0.10658341292353608, '11 and reported prevalence of chemosensory dysfunction from 5,1 % to 19.4 %']
[0.10720075977744738, 'under this program , vaccination levels reached 80 %']
[0.10786819792913292, 'in this study , the authors reported an accuracy of 92.6 %']

2
Related sentence to --  the 

We can see that some of the similar sentence are showing relevant results.But  we have initialized parameters randomly .If we can train, it will perform better

I have used two similarity metric here.
1. cosine similarity : which use cosine distance between two vector. which just check angel between two vector 

2. Normalized euclidean distance: which finds euclidean distance between two vectors, but vectors are normalized. As vectors are normalized, it does not matter the norm of the orginal vector is bigger or small.

So, if we want to find related sentence (if it does not matter length of the sentence and all) then cosine can be a better option than other metric. But if we normalized the vectors,then euclidean distance will also give similar results. We also have seen that thing.

What I have seen that both the distance metric have been given similar result. Though my RNN model is not trained. So, but parformance of each metric are same. 